In [5]:
import pandas as pd
import numpy as np
import ast
import re
from dataloader.datasets.dataset_csv import extract_label
from BIDS.core.vert_constants import v_name2idx, v_idx2name


# Original strings
original_strings = [
    "media/DATA/martina_ma/cutout_clean/ctfu00706/ctfu00706_9_subreg_cropped_cleaned.nii.gz",
    # Add more strings if needed
]

# Replace "clean" with "corpus" and "cleaned" with "corpus" in each string
modified_strings = [
    s.replace("clean", "corpus").replace("cleaned", "corpus")
    for s in original_strings
]

# Output the modified strings
for s in modified_strings:
    print(s)

path = "fractured_train_val_set.csv"

dataset = pd.read_csv(path, sep=",", index_col=0)
for index, row in dataset.iterrows():
    f = row.file_path.replace("cleaned", "corpus").replace("clean", "corpus").replace("_cropped", "")
    print(f)
    dataset.at[index,"file_path"] = f

dataset.to_csv("corpus_train_val_set.csv")

    


media/DATA/martina_ma/cutout_corpus/ctfu00706/ctfu00706_9_subreg_cropped_corpused.nii.gz
/media/DATA/martina_ma/cutout_corpus/verse599/verse599_ 8_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00015/ctfu00015_12_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00767/ctfu00767_13_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/fxclass0311/fxclass0311_15_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00558/ctfu00558_23_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00211/ctfu00211_14_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/fxclass0202/fxclass0202_11_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/verse096/verse096_17_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00903/ctfu00903_17_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00675/ctfu00675_16_subreg_corpus.nii.gz
/media/DATA/martina_ma/cutout_corpus/ctfu00500/ctfu00500_14_subreg_corpus.nii.gz
/media/DATA/mart

In [2]:

  
def extract_subject(input_string, sub):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(sub+r'\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None

def extract_ctfu(input_string):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(r'ctfu\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None


def parse_string(s,sub):
    try:
        # Safely evaluate the string as a dictionary
        dictionary = ast.literal_eval(s)

        # Extract and return the keys as a list
        keys_list = list(dictionary.keys())
        return dictionary #keys_list
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the input string: {e, sub}")
        return []
    
path = "/media/DATA/martina_ma/train_val_cleaned_seg_fails.csv"
data_list = []


dataset = pd.read_csv(path, sep=",")
dataset['fracture_flag'] = 'U'
def add_F(dataset, df_path = '/media/DATA/martina_ma/fxclass_bs_anomaly_cleaned.csv'):
    fxclass_bs_df =  pd.read_csv(df_path, delimiter=";", header = None)

    for index, row in fxclass_bs_df.iterrows():
        subject_full = row[0].split('_')[0]
        subject = subject_full.split('-')[1]
        matched_row = dataset.loc[dataset['file_path'].str.contains(subject)]
        if matched_row.empty:
            continue
        for m_row in matched_row.values:
            sub, label = extract_label(m_row[1])
            fractures_dict = parse_string(row[1],sub)
            label = v_idx2name.get(int(label),-1)
            idx = int(m_row[0])
            if len(fractures_dict) == 0 :
                dataset.at[idx,"fracture_flag"] = 'NF'
                continue
            value = fractures_dict.get(label,-1)
            if value == -1:
                dataset.at[m_row[0],"fracture_flag"] = 'NF'
                continue
            if 'F' in value:
                dataset.at[m_row[0],"fracture_flag"] = 'F'
            elif 'Metal' in value:
                dataset.at[m_row[0],"fracture_flag"] = 'NF' #because metal implant
    return dataset

d = add_F(dataset)
d_ctfu = add_F(d, "/media/DATA/martina_ma/ctfu_bs_anomaly.csv")

d_ctfu.to_csv('test_adding_F_ctfu.csv')   


Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('<unknown>', 1, 2, '{all ambig}', 1, 11)), 'fxclass0293')
Error parsing the input string: (SyntaxError('invalid syntax. Perhaps you forgot a comma?', ('

In [2]:
import numpy as np
import torch
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map
from dataloader.dataset_factory import get_data_loader, get_dataset
from torch.utils.data import DataLoader
from utils import arguments
import re
from torch.utils.data import Dataset
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
#from BIDS import NII
from BIDS import NII
from BIDS.core.np_utils import np_map_labels, Label_Map
import re
from BIDS.core.np_utils import np_volume
from dataloader.datasets.dataset_csv import model_map_to_segmentation_map, segmentation_map_to_model_map


####################################
/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/traitlets/config/application.py", line 1053, in launch_instance
    app.start()
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io

In [ ]:

def remove_labelsUntil10or7(dataset):
    data_list = []
    labels_to_exclude = [1,2,3,4,5,6,7]#evtl 8,9,10
    for index, row in dataset.iterrows():
        sub, label = extract_label(row.file_path)
        if int(label) not in labels_to_exclude:
            data_list.append(row)
    return data_list

    
path = "/media/DATA/martina_ma/train_val_cleaned_seg_fails.csv"

d_ctfu = pd.read_csv(path, sep=",")
datalist_filtered = remove_labelsUntil10or7(d_ctfu) 
dataframe = pd.DataFrame(datalist_filtered)
print(dataframe.head())
df = dataframe.drop(['Unnamed: 0'],axis=1)
print(df.head())


In [4]:

df.to_csv('test_clean_train_val.csv')   
df =  pd.read_csv('test_clean_train_val.csv', delimiter=",",index_col=0)
print(df.head())


                                           file_path  Split
0  /media/DATA/martina_ma/cutout/ctfu01098/ctfu01...  train
1  /media/DATA/martina_ma/cutout/ctfu00453/ctfu00...  train
2  /media/DATA/martina_ma/cutout/verse765/verse76...  train
3  /media/DATA/martina_ma/cutout/ctfu00996/ctfu00...  train
4  /media/DATA/martina_ma/cutout/ctfu01081/ctfu01...  train


In [3]:

opt = arguments.DAE_Option().get_opt(None, None)


dataset_train = get_dataset(arguments.DataSet_Option,"train")
dataset_val = get_dataset(arguments.DataSet_Option,"val")
print(len(dataset_train))
#part_tr = torch.utils.data.random_split(dataset, [0, len(dataset)-10])[0]
#subset_indices = range(100)#[0,1,2,3,4,5,6,7,8,9]
part_tr = dataset_train #torch.utils.data.Subset(dataset, subset_indices)
#datset = dataset.
train_dataloader = DataLoader(
        part_tr,
        batch_size=1,#opt.batch_size,
        #sampler=sampler,
        # with sampler, use the sample instead of this option
        shuffle=False,# if sampler else shuffle,
        num_workers=16,#opt.num_cpu,
        pin_memory=True,
        #drop_last=drop_last,
        #multiprocessing_context=get_context("fork"),
        persistent_workers=True,
    )
val_dataloader = DataLoader(
        dataset_val,
        batch_size=1,#opt.batch_size,
        #sampler=sampler,
        # with sampler, use the sample instead of this option
        shuffle=False,# if sampler else shuffle,
        num_workers=16,#opt.num_cpu,
        pin_memory=True,
        #drop_last=drop_last,
        #multiprocessing_context=get_context("fork"),
        persistent_workers=True,
    )

def calc_volume(seg_arr):
    seg = model_map_to_segmentation_map(seg_arr)
    labels = np.unique(seg_arr)
    labels = model_map_to_segmentation_map(labels)[1:]
    vol_dict = np_volume(seg, labels)
    total_volume = sum(vol_dict.values())
    return total_volume


test_clean_train_val.csv
test_clean_train_val.csv
17402


In [4]:
def extract_embeddings(train_dataloader, val_dataloader):
    data_list = []

    with torch.no_grad():
        for x in train_dataloader:
            img = x['img']
            label = x['label']
            subject = x['subject']
            label_str = str(label)
            subject_str = str(subject)
            frac = x['fracture']
            volume = calc_volume(img.cpu().numpy())

            data_item = {
                "label": label_str,
                "subject": subject_str,
                "fracture": frac,
                "volume": volume
            }

            data_list.append(data_item)
        for x in val_dataloader:
            img = x['img']#.to(device)
            label = x['label']
            subject = x['subject']
            frac = x['fracture']
            label_str = str(label)
            subject_str = str(subject)
            volume = calc_volume(img.cpu().numpy())

            data_item = {
               
                "label": label_str,
                "subject": subject_str,
                "fracture": frac,
                "volume": volume
            }

            data_list.append(data_item)

        
    # Convert the list of dictionaries to a DataFrame
    dataframe = pd.DataFrame(data_list)

    return dataframe


# Example usage:
vol_df = extract_embeddings(train_dataloader, val_dataloader)


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/media/DATA/martina_ma/dae/dataloader/datasets/dataset_csv.py", line 52, in __getitem__
    item = self.get_item_3D(from_im, folder, label, row.file_path)#, row.fracture_flag)
  File "/media/DATA/martina_ma/dae/dataloader/datasets/dataset_csv.py", line 113, in get_item_3D
    from_im = self.transform(from_im)
  File "/home/martina/.c

In [ ]:
import plotly.express as px
# pattern_shape="fracture_flag", category_orders=dict(label_reg=["C1", "C2", "C3", "C4","C5", "C6", "C7", "T1", "T2", "T3", "T4", "T5", "T6","T7", "T8", "T9", "T10", "T11", "T12","T13", "L1","L2", "L3","L4", "L5","L6"]
fig = px.histogram(vol_df, x="volume",color="fracture")
fig.update_layout(bargap=0.2)
fig.show()

In [ ]:

def add_ctfu_bs_anomaly_data(emb_df, filepath = "/media/DATA/martina_ma/ctfu_bs_anomaly.csv"):
    df = pd.read_csv(filepath, delimiter=";", header= None)
    for index, row in emb_df.iterrows():
        subject = row['subject'].strip("[]").replace("'", "")
        val = df[0].values
        arr = [subject in s for s in val]
        if not any(arr):
            continue
        matching_rows = df[0][np.array(arr)]
        for subject_full in matching_rows:
            selected_row = df.loc[df[0] == subject_full].iloc[0]#df['ID'] == subject
            fractures_dict = parse_string(selected_row[1])
            if fractures_dict == []:
                emb_df.at[index,"fracture_flag"] = 'NF'
                emb_df.at[index,"Implant"] = 'NM'
                emb_df.at[index,"fracture_grading"] = 0
                continue
            print("fracture dict exists")
            label = v_idx2name.get(int(row['label'].strip("[]").replace("'", "")),-1)
            if fractures_dict == -1:
                emb_df.at[index,"fracture_flag"] = 'NF'
                emb_df.at[index,"Implant"] = 'NM'
                continue
            print("fracture dict exists")
            value = fractures_dict.get(label,-1)
            if value == -1:
                emb_df.at[index,"fracture_flag"] = 'NF'
                emb_df.at[index,"Implant"] = 'NM'
                continue
            if 'F' in value:
                emb_df.at[index,"fracture_flag"] = 'F'
                emb_df.at[index,"fracture_grading"] = int(re.search(r'\d+', value).group())
                emb_df.at[index,"Implant"] = 'NM'
            elif 'Metal' in value:
                emb_df.at[index,"Implant"] = 'M'
                emb_df.at[index,"fracture_flag"] = 'NF'
    return emb_df
    
 

In [11]:
import pandas as pd
import numpy as np
import ast
import re
from BIDS.core.vert_constants import v_name2idx, v_idx2name

def compare_fracture_annotation():
    df_path = pd.read_excel("/media/DATA/martina_ma/pathologic_fractures_clean.xlsx", sheet_name='data')
    df_bs = pd.read_csv("/media/DATA/martina_ma/fxclass_bs_anomaly.csv", delimiter=";", header = None)

    for index, row in df_path.iterrows():
        subject = row['ID'].strip("[]").replace("'", "")
        val = df_bs[0].values
        arr = [subject in s for s in val]
        if not any(arr):
            continue
        matching_rows = df_bs[0][np.array(arr)]
        for subject_full in matching_rows:
            selected_row = df_bs.loc[df_bs[0] == subject_full].iloc[0]
            fractures_dict = parse_string(selected_row[1])
            if len(fractures_dict) == 0 :
               # print("no fractures for subject"+subject_full)
                continue
            if row['main_fracture'] not in fractures_dict:
                keys = ",".join(list(fractures_dict.keys()))
                print("discrepancy: "+subject_full+" for fracture " +row['main_fracture'] + " not in ["+keys+"]") #keys = "".join(list(dict.keys()))
    return 



def add_pathologic_fractures_clean_data(emb_df, filepath = '/media/DATA/martina_ma/pathologic_fractures_clean.xlsx'):
    df = pd.read_excel(filepath, sheet_name='data')

    # emb_df["fracture_flag"] = 'U'
    # emb_df["sex"] = 'U'
    # emb_df["age"] = -1
    # emb_df["fracture_heights"] = None
    # emb_df["exam_date"] = -1
    # emb_df["cancerous"] = -1
    for index, row in emb_df.iterrows():
        subject = row['subject'].strip("[]").replace("'", "")
        #print(df['ID'])
        if subject not in df['ID'].values:
            continue
        #print('included')
        selected_row = df.loc[df['ID'] == subject].iloc[0]#df['ID'] == subject
        emb_df.at[index,'cancerous'] = selected_row['label']
        emb_df.at[index, 'sex'] = selected_row['sex']
        emb_df.at[index, 'age'] = selected_row['age']
        emb_df.at[index,"exam_date"] = selected_row['exam_date']
        fh = [value for value in  selected_row['fracture_heights'].split('_')]#.replace(" ", "")]
        emb_df.at[index, 'fracture_heights'] = [v_name2idx.get(f.replace(" ", ""), -1) for f in fh]
        x = emb_df.at[index, 'fracture_heights']
        label = int(row['label'].strip("[]").replace("'", ""))
        mf = selected_row['main_fracture'].replace(" ", "")
        if label == v_name2idx.get(mf, -1): #evtl replace wieder hinter main_fra
            emb_df.at[index,"fracture_flag"] = 'F'
        elif label in emb_df['fracture_heights']:
            emb_df.at[index,"fracture_flag"] = 'FH'
        else:
            emb_df.at[index,"fracture_flag"] = 'NF'
    return emb_df


def add_fxclass_bs_anomaly_data(emb_df, filepath = "/media/DATA/martina_ma/fxclass_bs_anomaly.csv"):
    df = pd.read_csv(filepath, delimiter=";", header = None)
    # emb_df["fracture_grading"] = -1
    # emb_df["Implant"] = 'U'
    for index, row in emb_df.iterrows():
        subject = row['subject'].strip("[]").replace("'", "")
        val = df[0].values
        arr = [subject in s for s in val]
        if not any(arr):
            continue
        matching_rows = df[0][np.array(arr)]
        for subject_full in matching_rows:
            selected_row = df.loc[df[0] == subject_full].iloc[0]
            fractures_dict = parse_string(selected_row[1])
            #print(type(anomaly_id))
            #print(anomaly_id)
            label = v_idx2name.get(int(row['label'].strip("[]").replace("'", "")),-1)
            if len(fractures_dict) == 0 :
                emb_df.at[index,"fracture_flag"] = 'NF'
                emb_df.at[index,"Implant"] = 'NM'
                emb_df.at[index,"fracture_grading"] = 0
                print("no fractures")
                continue
            print("fracture dict exists")
            value = fractures_dict.get(label,-1)
            if value == -1:
                emb_df.at[index,"fracture_flag"] = 'NF'
                emb_df.at[index,"Implant"] = 'NM'
                continue
            if 'F' in value:
                emb_df.at[index,"fracture_flag"] = 'F'
                emb_df.at[index,"fracture_grading"] = int(re.search(r'\d+', value).group())
                emb_df.at[index,"Implant"] = 'NM'
            elif 'Metal' in value:
                emb_df.at[index,"Implant"] = 'M'
                emb_df.at[index,"fracture_flag"] = 'NF' #because metal implant

    return emb_df




def extract_subject(input_string, sub):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(sub+r'\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None

def extract_ctfu(input_string):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(r'ctfu\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None


def parse_string(s):
    try:
        # Safely evaluate the string as a dictionary
        dictionary = ast.literal_eval(s)

        # Extract and return the keys as a list
        keys_list = list(dictionary.keys())
        return dictionary #keys_list
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the input string: {e}")
        return []
    
compare_fracture_annotation()

discrepancy: sub-fxclass0050_dir-sag_sequ-Wirbelsaule20mmsag3 for fracture L1 not in [T11,L2]
discrepancy: sub-fxclass0122_dir-iso_sequ-HufteKFi2iDose2501 for fracture L2 not in [T6,T10,L3,L5]
discrepancy: sub-fxclass0148_dir-sag_sequ-sagBWSLWSKN80252 for fracture T1 not in [T9,T12,L3]
discrepancy: sub-fxclass0150_dir-sag_sequ-WS30MPRsag4 for fracture L5 not in [T10]
discrepancy: sub-fxclass0157_dir-iso_sequ-WSKFi2iDose2201 for fracture T6 not in [T8,T10]
discrepancy: sub-fxclass0174_dir-sag_sequ-WS30MPR4 for fracture L3 not in [T11,T12,L2]
discrepancy: sub-fxclass0181_dir-cor_sequ-MPRTKWF3mmcor6 for fracture L2 not in [L4]
discrepancy: sub-fxclass0201_dir-iso_sequ-WSKFi2iDose2301 for fracture T9 not in [L1]
discrepancy: sub-fxclass0221_dir-sag_sequ-MPRWSKFsag3mmSAF6 for fracture T7 not in [T4,T6,T10,T11,L1,L4]
discrepancy: sub-fxclass0233_dir-sag_sequ-Wirbelsaule20mmsag8 for fracture T2 not in [L2]
discrepancy: sub-fxclass0256_dir-iso_sequ-WSKFfurMPRiDose4201 for fracture L3 not in [L

du kannst NII.get_segmentation_connected_components machen
oder falls du es zu dem zeitpunkt als array hast, unter BIDS.core.np_utils dieselbe funktion np_connected_components
daraus erhälst du ein
subreg_cc, subreg_cc_stats =  (die funktion von oben)
cc_size_threshold: int = 20 (check das vllt. bei dem frakturierten und drüber, was hier ne gute zahl)
result_arr = input.copy() (kopier dein input, sobalds ein arr ist)
labels =  (deine möglichen subregion labels, wie sie halt grad vorliegen
for l in labels:
   cc_indices = [i for i, v in enumerate(subreg_cc_stats[label]["voxel_counts"]) if v < cc_size_threshold] #yields cc indices of cc volume below threshold
   for cc_idx in cc_indices:
      mask_cc = subreg_cc[label]
      mask_cc_l = mask_cc.copy()
      mask_cc_l[mask_cc_l != cc_idx] = 0
      result_arr[mask_cc_l != 0] = 0
return result_arr


In [26]:
from pathlib import Path
import nibabel as nib
from BIDS import NII
import numpy as np
from BIDS.core.np_utils import np_map_labels, Label_Map, np_connected_components
from dataloader.datasets.dataset_csv import model_map_to_segmentation_map, segmentation_map_to_model_map 
#3D
def load_nifti(nii):
        
    
    nii = NII.load(
    "/media/DATA/martina_ma/cutout/fxclass0149/fxclass0149_012_subreg_cropped.nii.gz",#"" "/media/DATA/martina_ma/cutout/fxclass0288/fxclass0288_019_subreg_cropped.nii.gz",
        True,
    )
    return nii, nii.get_seg_array()

In [28]:
# du kannst NII.get_segmentation_connected_components machen
# oder falls du es zu dem zeitpunkt als array hast, unter BIDS.core.np_utils dieselbe funktion np_connected_components
# daraus erhälst du ein
#import cc3d as c
def clean(nii_seg_arr, nii):
   labels =  [0,41,42,43,44,45,46,47,48,49,50]
   subreg_cc, subreg_cc_stats = nii.get_segmentation_connected_components( labels, connectivity = 3)#np_connected_components(nii_seg_arr) # (die funktion von oben)
   cc_size_threshold: int = 30 #(check das vllt. bei dem frakturierten und drüber, was hier ne gute zahl)
   result_arr = nii_seg_arr.copy() #(kopier dein input, sobalds ein arr ist)
   labels =  [0,41,42,43,44,45,46,47,48,49,50]#(deine möglichen subregion labels, wie sie halt grad vorliegen
   for l in labels:
      cc_indices = [i for i, v in enumerate(subreg_cc_stats[l]["voxel_counts"]) if v < cc_size_threshold] #yields cc indices of cc volume below threshold
      for cc_idx in cc_indices:
         mask_cc = subreg_cc[l]
         mask_cc_l = mask_cc.copy()
         mask_cc_l[mask_cc_l != cc_idx+1] = 0
         result_arr[mask_cc_l != 0] = 0#!= 0] = 0
   return result_arr
nii, nii_seg_arr = load_nifti('nii')
print(np.unique(nii_seg_arr))
result_arr = clean(nii_seg_arr,nii)
nii.set_array_(result_arr).save("Cleaned_fxclass0149_012_30.nii.gz")

[ 0 41 42 43 44 45 46 47 48 49 50]
[ ] 'set_array' with different dtype: from uint8 to uint16
[*] Save Cleaned_fxclass0149_012_30.nii.gz as uint8


In [ ]:
path = "/media/DATA/martina_ma/train_val_cleaned_seg_fails.csv"
dataset = pd.read_csv(path, sep=",")

In [2]:
import pandas as pd
import numpy as np
import ast
import re
from dataloader.datasets.dataset_csv import extract_label
from BIDS.core.vert_constants import v_name2idx, v_idx2name
  
def extract_subject(input_string, sub):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(sub+r'\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None

def extract_ctfu(input_string):
    # Define a regular expression pattern to match "fxclassXYZ"
    pattern = re.compile(r'ctfu\d+')

    # Use search to find the first match in the input string
    match = pattern.search(input_string)

    # Return the matched string or None if no match is found
    return match.group() if match else None


def parse_string(s):
    try:
        # Safely evaluate the string as a dictionary
        dictionary = ast.literal_eval(s)

        # Extract and return the keys as a list
        keys_list = list(dictionary.keys())
        return dictionary #keys_list
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the input string: {e}")
        return []


def count_T123( filepath = "/media/DATA/martina_ma/ctfu_bs_anomaly.csv"):
    df = pd.read_csv(filepath, delimiter=";", header = None)
    count_t1_f = 0
    count_t2_f = 0
    count_t3_f = 0
    count_t1_m = 0
    count_t2_m = 0
    count_t3_m = 0
    # emb_df["fracture_grading"] = -1
    # emb_df["Implant"] = 'U'
    for index, row in df.iterrows():
        fractures_dict = parse_string(row[1])
            #print(type(anomaly_id))
            #print(anomaly_id)
            #label = v_idx2name.get(int(row['label'].strip("[]").replace("'", "")),-1)
        if len(fractures_dict) == 0 :
            continue
            print("fracture dict exists")
        value_T1 = fractures_dict.get('T1',-1)
        value_T2 = fractures_dict.get('T2',-1)
        value_T3 = fractures_dict.get('T3',-1)
        values = [value_T1,value_T2,value_T3]
        for v in values:
            if v != -1:
            # if value == -1:
                    # emb_df.at[index,"fracture_flag"] = 'NF'
                    # emb_df.at[index,"Implant"] = 'NM'
                    #continue
                if 'F' in v:
                    count_t1_f = count_t1_f +1
                elif 'Metal' in v:
                    count_t1_m = count_t1_m +1
                #emb_df.at[index,"fracture_flag"] = 'NF' #because metal implant

    return count_t1_f, count_t1_m

print(count_T123())

/home/martina/.conda/envs/dae_cond/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(15, 19)
